In [2]:
from scipy import sparse
from scipy.sparse import csr_matrix
import numpy as np

In [6]:
#np.random.rand(1000, 1000)
A = sparse.rand(1000,1000).tocsr()
#A = sparse.rand(60000,60000, density=0.001).tocsr()

print(len(A.data))
print(A.shape)

10000
(1000, 1000)


In [5]:
%timeit A.getnnz(axis=0).nonzero()[0]
%timeit set(A.nonzero()[1])

The slowest run took 4.44 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 24.1 µs per loop
1000 loops, best of 3: 726 µs per loop


this may be helpful:
https://stackoverflow.com/questions/19784868/what-is-most-efficient-way-of-setting-row-to-zeros-for-a-sparse-scipy-matrix?noredirect=1&lq=1

I may be able to set rows/cols to 0 instead of getting a submatrix. 
If I sorted the matrix by putting all of the nodes in N at the beginning, then I would just set everything else to 0
Sorting the matrix could be expensive though...

In [58]:
def get_neighbors(A):
    #neighbors = np.arange(0,A.shape[0])
    all_neighbors = np.array([])
    for n in range(0,A.shape[0]):
#         neighbors[n] = A.indices[A.indptr[n]:A.indptr[n+1]]
        neighbors = A.indices[A.indptr[n]:A.indptr[n+1]]
        all_neighbors = np.append(all_neighbors, neighbors)
    return all_neighbors
      
neighbors = get_neighbors(A)
print(neighbors)
print(len(neighbors))
#%timeit get_neighbors(A)

[ 18.  74. 229. ... 330. 445. 615.]
10000


In [59]:
Al = A.tolil()

In [61]:
#Al.nonzero()
def get_neighbors2(Al):
    all_neighbors = np.array([])
    for n in range(0,A.shape[0]):
        np.append(all_neighbors, Al.getrowview(n).nonzero()[1])
    return all_neighbors
        
%timeit get_neighbors2(Al)
# neighbors = get_neighbors(A)
# print(len(all_neighbors))
# print(len(all_neighbors[0]))

1 loop, best of 3: 163 ms per loop


In [107]:
def get_neighbors3(Al):
    all_neighbors = [[] for k in range(Al.shape[0])]
    rows, cols = Al.nonzero()
    for i in range(len(rows)):
        all_neighbors[rows[i]].append(cols[i])
    all_neighbors = np.array([np.array(x) for x in all_neighbors])
    return all_neighbors

def get_neighbors4(Al):
    all_neighbors = []
    rows, cols = Al.nonzero()
    prev_i = 0
    for i in rows:
        if i != prev_i:
            all_neighbors.append(cols[prev_i:i-1])
            prev_i = i
    all_neighbors.append(cols[prev_i:i-1])
    return all_neighbors

%timeit get_neighbors3(A)
%timeit get_neighbors4(Al)

100 loops, best of 3: 4.16 ms per loop
100 loops, best of 3: 5.71 ms per loop


In [106]:
n3 = get_neighbors3(Al)
n4 = get_neighbors4(Al)
print(len(n3))
print(len(n4))
print np.array([[1,2,3],[2,3,4],[3,4,5]])
# print np.asarray(n4)

1000
1000
[[1 2 3]
 [2 3 4]
 [3 4 5]]


In [108]:
A.nonzero()

(array([  0,   0,   0, ..., 999, 999, 999], dtype=int32),
 array([ 18,  74, 229, ..., 330, 445, 615], dtype=int32))

In [89]:
all_neighbors = []*Al.shape[0]
all_neighbors

[]

In [87]:
test = np.array([[], []])
print(test[0])
test[0] = np.asarray([1,2,3])
test[0] 

[]


ValueError: could not broadcast input array from shape (3) into shape (0)

In [116]:
# NumPy arrays
a = np.unique(np.random.randint(0, high=1000, size=1000))
b = np.unique(np.random.randint(0, high=1000, size=1000))

# Python's built-in set objects
sa = set(a)
sb = set(b)

In [126]:
%timeit np.intersect1d(a, b, assume_unique=True)
%timeit sa & sb

10000 loops, best of 3: 22.9 µs per loop


In [2]:
# compare using a lil matrix
x=sparse.lil_matrix(np.arange(120).reshape(24,5))

In [5]:
%timeit x[10,:]=10
%timeit y=x.getrowview(10);y[:]=10
%timeit x[10,:]
%timeit x.getrowview(10)

The slowest run took 5.24 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 71.8 µs per loop
10000 loops, best of 3: 96.7 µs per loop
10000 loops, best of 3: 51 µs per loop
10000 loops, best of 3: 24.2 µs per loop


I already tried speeding-up ripple using a lil matrix to index, and then convert that to a csr matrix.
Didn't help!

In [26]:
A = sparse.rand(1000,10000).tocsr()
B = A.tolil()
C = A.tocsc()
#A = sparse.rand(60000,60000, density=0.001).tocsr()

print(len(A.data))
print(A.shape)

100000
(1000, 10000)


In [24]:
def get_rows_lil(A, rows):
    test = []
    for i in rows:
#         row = A.getrowview(i)
#         test.append((row > .5).nonzero()[1])
        test.append(A.rows[i])
    return test
        
def get_rows_csr(A, rows):
    test = []
    for i in rows:
        row = A[i]
        test.append((row > .5).nonzero()[1])
    return test

def get_rows_csr2(A, rows):
    test = []
    for i in rows:
        row = A[i].toarray().flatten()
        test.append(np.where(row > 0)[0])
    return test

In [6]:
%timeit get_rows_lil(B, range(B.shape[0]))
%timeit get_rows_csr(B, range(B.shape[0]))
%timeit get_rows_csr(C, range(C.shape[0]))

100 loops, best of 3: 14.5 ms per loop
10 loops, best of 3: 35.3 ms per loop
1 loop, best of 3: 226 ms per loop


In [27]:
%timeit get_rows_lil(B, range(B.shape[0]))
%timeit get_rows_csr(B, range(B.shape[0]))
%timeit get_rows_csr(C, range(C.shape[0]))
D = C.todok()
%timeit get_rows_csr(D, range(C.shape[0]))
# not implemented for bsr
# E = C.tobsr()
# %timeit get_rows_csr(E, range(C.shape[0]))
%timeit get_rows_csr2(C, range(C.shape[0]))

10000 loops, best of 3: 170 µs per loop
1 loop, best of 3: 286 ms per loop
1 loop, best of 3: 1.02 s per loop
1 loop, best of 3: 7.99 s per loop
1 loop, best of 3: 1.28 s per loop


In [23]:
B.rows[0]

[22, 45, 91, 296, 348, 443, 492, 578, 734, 743, 969]

In [62]:
# try indexing a row of scipy
rows = np.random.randint(1000, size=10)
cols = np.random.randint(1000, size=10)

# getrowview doesn't actually do anything special
# B.getrowview(rows)
# B[rows,:][:,cols]
# %timeit B.getrowview(rows)[cols]
# %timeit C[rows][:,cols]
# %timeit C[:,cols][rows]
%timeit B[rows][:,cols]
%timeit A[rows][:,cols]
# %timeit B[0]
# %timeit B.getrowview(0)

10000 loops, best of 3: 158 µs per loop
1000 loops, best of 3: 663 µs per loop


In [59]:
def get_rows_lil(B, rows):
    new = sparse.lil_matrix(len(rows), B.shape[1])
    new.rows = B.rows[rows]
    new.data = B.data[rows]
    return new

In [69]:
# print(B.shape)
%timeit get_rows_lil(B,rows)
%timeit B[rows]

The slowest run took 4.61 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 265 µs per loop
10000 loops, best of 3: 57.7 µs per loop


# Ripple Python vs Matlab

The bottleneck for ripple is currently getting the submatrix at each step.
Try comparing the time it takes to get a submatrix in python and in matlab

In [14]:
# these are about the dimensions of the network I'm working with. 70K nodes and 5M edges
A = sparse.rand(70000,70000, density=0.001).tocsr()
B = A.tolil()
C = A.tocsc()

print(len(A.data))
print(A.shape)

4900000
(70000, 70000)


In [12]:
# try getting a subset of nodes from the matrix
nodes = np.random.randint(70000, size=1000)

In [17]:
%timeit test = A[nodes][:,nodes]
%timeit test = B[nodes][:,nodes]
%timeit test = C[nodes][:,nodes]

100 loops, best of 3: 1.72 ms per loop
10 loops, best of 3: 62.4 ms per loop
10 loops, best of 3: 23.7 ms per loop


In [15]:
x = np.random.rand(70000)

In [12]:
# import time
# start = time.time()
%timeit sparse.csr_matrix.dot(A,x)
# print(time.time() - start)

100 loops, best of 3: 12.5 ms per loop


### Matlab

In [ ]:
# now repeat the same in matlab
A = sprand(70000,70000,.001);
Atrans = A';
nodes = randi(70000, [1,1000]);
f = @() A(nodes, nodes);
timeit(f)
x = rand(70000,1);
f = @() Atrans'*x;
timeit(f)

### Julia
I wonder if I could use Julia? That is supposedly fast.
Running the code below, Julia takes about 0.0013 sec (1.13 ms)

In [ ]:
# I was able to repeat the same test in Julia with the following code:
# first run julia: /data/jeff-law/tools/julia/julia-1.0.0/bin/julia
using SparseArrays
using BenchmarkTools
A = sprand(Float64, 70000,70000,.001)
nodes = rand(1:70000, 1000)
@benchmark A[nodes,nodes]

- Matlab takes .......  ~8.2e-04 sec
- Python takes ........ 1.7e-03 sec
- Julia takes ............ 1.1e-03 sec

Looks like Matlab is about 2x faster than python for sparse matrix slicing

here's a helpful link: 
https://stackoverflow.com/questions/45516690/is-sparse-matrix-vector-multiplication-faster-in-matlab-than-in-python

For matrix multiplication A*x:


| Method | Time (sec) | notes |
|--------|------ | ----- |
|Matlab  | 0.0071 | Atrans'*x is fastser for some reason |
| Python | 0.012  | |
| Julia  | 0.018  | |

I'm not totally sure if Matlab and Julia are utilizing the sparse matrix, but I can't find any documentation for how to do sparse dot product in Matlab and Julia


In [2]:
# extra networkx comparison
import networkx as nx
G = nx.Graph()
G.add_path(range(1000))
def test1(G):
    for u,v, in G.edges():
        G.edges[u,v]['weight'] = 2
def test2(G):
    for u,v, in G.edges():
        G.adj[u][v]['weight'] = 3
%timeit test1(G)
%timeit test2(G)

100 loops, best of 3: 1.89 ms per loop
1000 loops, best of 3: 1.4 ms per loop


In [18]:
from scipy.sparse import linalg as spla
# also test different solvers
M2 = spla.spilu(C)
M2 = spla.LinearOperator(C.shape, M2.solve)

In [ ]:
%timeit spla.cg(A, x)
%timeit spla.cg(A, x, M=M2)

1 loop, best of 3: 3h 27min 21s per loop


/home/jeffl/.local/lib/python2.7/site-packages/scipy/sparse/linalg/isolve/iterative.py:308: RuntimeWarning: invalid value encountered in multiply
  work[slice2] *= sclr2
